In [ ]:
"""
Docstring

This code prepares a dataset for Naive Bays classification in excel. The program identifies null values in the data set
replaces them with the appropriate data and gets dummies for categorical variables. Finally the program splits the data
into training and testing sets, writing them to new excel files for further analysis in excel. 

Bugs/errors
Our program is not able to recognize missing values in all features. 

"""



# importing all the Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split

# importing file by specifying the location with the variable us_census
file = './james.csv'

# labeling columns
column_names = ["Age", 
                "Job_occupation", 
                "Type_of_employment", 
                "Number_of_hours_working_per_week",
                "Capital_gain_loss",
                "Level_of_education", 
                "Completed_years_of_education", 
                "Marital_status",
                "Relationship_inside_the_household", 
                "Race", 
                "Gender", 
                "Country_of_birth",
                "Income_per_year"]

# setting data types
data_types = {'Job_occupation': str,
             'Type_of_employment': str,
             'Level_of_education' : str,
             'Marital_status': str,
             'Relationship_inside_the_household' : str,
             'Race' : str,
             'Gender' : str,
             'Country_of_birth': str}

# reading file with added columns, data types and recognizing " ?" as NaN
us_census = pd.read_csv(file, 
                        names=column_names, 
                        dtype = data_types, 
                        na_values = " ?")

# amount of rows with missing values
us_census.dropna(how = "any").shape

# creating missing value flags
us_census['m_Job_occupation'] = us_census['Job_occupation'].isnull().astype(int)
us_census['m_Type_of_employment'] = us_census['Type_of_employment'].isnull().astype(int)
us_census['m_Country_of_birth'] = us_census['Country_of_birth'].isnull().astype(int)  

# creating copy of data frame
us_census_dropped = pd.DataFrame.copy(us_census)

# dropping the null values from the DataFrame
us_census_dropped = us_census_dropped.dropna()


# replacing the null values with the mode of each variable
us_census.loc[:,"Job_occupation"].fillna(us_census_dropped['Job_occupation'].mode()[0], inplace = True)
us_census.loc[:,"Country_of_birth"].fillna(us_census_dropped['Country_of_birth'].mode()[0],inplace = True)
us_census.loc[:,"Type_of_employment"].fillna(us_census_dropped['Type_of_employment'].mode()[0] ,inplace = True)

     
        
# creating dummies for specific x variables
dummies_job = pd.get_dummies(drop_first = True, 
                             data = us_census.loc[:, [ "Race",
                                                      "Gender",
                                                     "Relationship_inside_the_household",]])           

# dropping the columns that are no longer needed from the original data
us_census = us_census.drop([
                            "Race",
                          "Gender",                 
                         "Relationship_inside_the_household",
                           'm_Job_occupation',
                            'm_Type_of_employment',
                            'm_Country_of_birth',],axis = 1)


# joining the dummies into the dataframe
us_census = us_census.join(dummies_job)


# preparing explanatory variable data
census_data   = us_census.drop(['Income_per_year'],
                                axis = 1)


# preparing response variable data
income_variable = us_census.loc[ : , 'Income_per_year']


# preparing training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
            census_data,
            income_variable,
            test_size = 0.20,
            random_state = 1223)



# merging x_train and y_train 
census_train = pd.concat([x_train, y_train], axis = 1)

# merging x_test and y_test 
census_test = pd.concat([x_test, y_test], axis = 1)

# saving the training set as an Excel file
census_train.to_excel(excel_writer = "census_train.xlsx",
                        index        = False)


# saving the testing set as an Excel file
census_test.to_excel(excel_writer = "census_test.xlsx",
                       index        = False)
